# Analisador de empresas

Tem como objetivo extrair informações relacionadas as empresas de uma determinada região e sua classificação no Google Meu Negócio e Facebook, se houver mídia relacionada.

In [61]:
!pip install psycopg2-binary

In [36]:
DEBUG = True

def LOG(funcao, mensagem, pulaLinhaAntes = False, pulaLinhaDepois = False):
    if DEBUG == True:
        if pulaLinhaAntes:
            print('\n');
        print('[{}] => {}'.format(funcao, mensagem))
        if pulaLinhaDepois:
            print('\n');

In [67]:
from urllib.request import urlopen
import json
import psycopg2

```sql
CREATE TABLE empresa (
    id VARCHAR(255) PRIMARY KEY,
    nome VARCHAR(255)
)
```

In [82]:
def inserirEmpresaBancoDados(empresa):

    LOG('inserirBancoDados', 'Iniciado')

    conn = psycopg2.connect(dbname="COLETADEDADOS", user="postgres", password="<senha>", host="localhost")

    cur = conn.cursor()

    cur.execute("INSERT INTO empresa (id, nome) VALUES (%s, %s)",(empresa['codigo'], empresa['nome']))

    conn.commit()

    cur.close()
    conn.close()

    LOG('inserirBancoDados', 'Finalizado')

In [79]:
def buscarEmpresasDaPagina(categoria, cidade, estado, pagina):

    LOG('buscarEmpresasDaPagina', 'Iniciado')

    site = 'https://guiafacil.com/buscaAjax.php?busca={}&cidade={}&tipoBusca=segmento&estado={}&pagina={}'
    site = site.format(categoria, cidade, estado, pagina)

    LOG('buscarEmpresasDaPagina', 'Buscar: {}'.format(site))

    conteudo = urlopen(site).read()
    conteudoJson = json.loads(conteudo)
    resultado = conteudoJson['resultados']

    LOG('buscarEmpresasDaPagina', 'Quantidade de empresas da página: {}'.format(len(resultado)))
    LOG('buscarEmpresasDaPagina', 'Finalizado')

    return resultado

In [80]:
def buscarEmpresas(categoria, cidade, estado):

    LOG('buscarEmpresas', 'Iniciado')

    resultado = []
    pagina = 1

    while(True):
        resultadoPagina = buscarEmpresasDaPagina(categoria, cidade, estado, pagina)
        if (resultadoPagina == 2):
            break;
        for empresa in resultadoPagina:
            inserirEmpresaBancoDados(empresa)

        resultado += resultadoPagina

    LOG('buscarEmpresas', 'Quantidade de empresas: {}'.format(len(resultado)))
    LOG('buscarEmpresas', 'Finalizado')

    return resultado

In [83]:
buscarEmpresas('', 'joinville', 'santa-catarina')

[buscarEmpresas] => Iniciado
[buscarEmpresasDaPagina] => Iniciado
[buscarEmpresasDaPagina] => Buscar: https://guiafacil.com/buscaAjax.php?busca=&cidade=joinville&tipoBusca=segmento&estado=santa-catarina&pagina=1
[buscarEmpresasDaPagina] => Quantidade de empresas da página: 20
[buscarEmpresasDaPagina] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => Iniciado
[inserirBancoDados] => Finalizado
[inserirBancoDados] => I

UniqueViolation: duplicate key value violates unique constraint "empresa_pkey"
DETAIL:  Key (id)=(1019716.2) already exists.


In [3]:
def getGuiaFacilCategories():
    print('\nGetGuiaFacilCategories started...\n')
    site = 'https://www.guiafacil.com/'
    print('Opening {}...'.format(site))
    html = urlopen(site)
    print('Parsing content...')
    bs = BeautifulSoup(html.read(),'html.parser')
    print('Find all categories...')
    divQuickleSearch = bs.find('div', { 'id': 'busca_rapida'})
    categories = divQuickleSearch.findAll('a', href=re.compile('^(?!http)(.)*$'))
    formattedCategories = []
    for category in categories:
        formattedCategories.append(unidecode(category['title'].lower().replace(' ', '-')))
    print('Categories found {}'.format(formattedCategories))
    print('\nGetGuiaFacilCategories finished...\n')
    return formattedCategories

In [4]:
def getCategoryPage(url, city, state):
    page = 'https://www.guiafacil.com/titulo/{}/{}/{}/'.format(url, city, state)
    try:
        html = urlopen(page)
        bs = BeautifulSoup(html.read(),'html.parser')
        divResults = bs.find('div', { 'id': 'resultados'})
    except Exception:
        divResults = 'Page {} not found!'.format(page)
    return divResults

In [ ]:
categories = getGuiaFacilCategories()
for category in categories:
    print(getCategoryPage(category, 'joinville', 'santa-catarina'))


GetGuiaFacilCategories started...

Opening https://www.guiafacil.com/...
Parsing content...


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Find all categories...
Categories found ['pizzarias', 'restaurantes', 'lanches', 'churrascaria', 'concessionarias', 'oficinas', 'autopecas', 'motos', 'materiais-de-construcao', 'moveis-sob-medida', 'arquitetos', 'marmores-e-granitos', 'clinicas-de-estetica', 'farmacias', 'hospitais', 'dentistas', 'eletricistas', 'serralheiros', 'fretes', 'informatica', 'supermercados', 'pet-shops', 'papelarias', 'imobiliarias']

GetGuiaFacilCategories finished...

